## RSA Cipher



### Key generation

* Choose two primes $p$ and $q$

* Compute $n=pq$

* Compute the least common multiple of $p-1$ and $q-1$, and call it $\lambda(n)=lcm(p-1,q-1)$

* Choose an integer $e$ coprime to $\lambda(n)$

* Compute the inverse $d$ of $e$ modulo $\lambda(n)$ 

Now, say that Alice wants to receive from Bob a message. 

* $(n,e)$ is a public key, which Alice sends to Bob through a reliable channel. 
* $(n,d)$ is a private key, which Alice keeps for herself. 




See [Wikipedia](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) for more details. See also [Khan Academy](https://www.khanacademy.org/computing/computer-science/cryptography#modern-crypt) for a greate video introduction. 


### Encryption and Decryption 

Bob translates his message M to an integer $m$, and then converts it to ciphertext using 

$$ c = m^e \pmod{n} $$

When Alices receives the value of 'c', she decodes it using 

$$ m = c^d \pmod{n}$$ 

which recovers $m$. 

### Examples

* Choose $p=61$, $q=53$ which are two prime numbers
* Compute $n=pq = 3233$, 
* Compute the least common multiple, 

$$ \lambda (3233) = lcm (60, 52) = 780 $$

* Choose between $1<e<780$ this is coprime to 780. Let $e=17$. 
* Compute $d$, the modular multiplicative inverse of $ 17 \pmod {780}$.  
   
   - $780=2^2*3*5*13$ 
   - $\phi(780) = (2-1)^2(3-1)(5-1)(13-1) = 96$
   - $ d= 17^{96-1} \pmod{780} = 413$

Now the public/encryption key is $(n=3233, e=17)$, the private/decryption key is $(n=3233, d=413)$. 

In this case, $p$ and $q$ are small, it is very easy to guess their values from $n$. In reality, large prime numbers are used such that the decomposition of $n$ to prime numbers is difficult. 

Now let's try the encryption and decryption 

Bob wants to send a message "HI", e.g., H=7, I=8. He first encodes it with the key he got from Alice $(n=3233, e=17)$,

$$ c1 = 7^{17} \pmod{3233} = 2369 $$
$$ c2 = 8^{17} \pmod{3233} = 2041 $$

He sends "2369, 2041" to Alice. 

Alice now uses her private key $(n=3233, d=413)$ to decode it, 

$$ m1 = 2369^{413} \pmod{3233} = 7 $$
$$ m2 = 2041^{413} \pmod{3233} = 8 $$

and she gets "7 8". With the alphabetic table, she understands the message is "HI". 



### Python routines 

See [RSACipher.py](RSACipher.py) for native implementations of all integer operations. For faster processing, especially for large numbers, use [RSACipherGMP.py](RSACipherGMP.py) which uses [GNU MP Library](https://gmplib.org/manual/) and [gmpy2](https://gmpy2.readthedocs.io) interface.

#### Key generation / Encrypt / Decrypt

In [1]:
# import my RSA python module
import RSACipher as rsa

# generate public and private key pairs
# bits limit the range of prime numbers, 2^bits[0], 2^bits[1]
# or (2^2, 2^8) here
public_key, private_key = rsa.generate_key(bits=(2, 8), debug=True)


# get n from the public_key
e, n = public_key 

# message to be sent 
plain = 1603 % n 
# encrypt by Bob with the Alice's public_key
cipher = rsa.encrypt(plain, public_key)
# Alice uses her private key to decrypt
decipher = rsa.decrypt(cipher, private_key)

print("plain :", plain, "cipher: ", cipher, "decrypted: ", decipher)

RSA Key Generator
p=  89  q=  149
lambda(n) = 3256
Public/Encryption Key  (e, n): (1171, 13261)
Private/Decryption Key (d, n): (1571, 13261)
plain : 1603 cipher:  2404 decrypted:  1603


#### Crack a RSA key

One way to crack RSA key is to factorize $n$ into $p$ and $q$, and $e$ can be easily found out. For a small $n$, it is possible, for example, by [Fermat's factorization method](https://en.wikipedia.org/wiki/Fermat%27s_factorization_method). 

In [2]:
p, q = rsa.fermat_factors(13631579)
print(p,q)

13 1048583


Of course. Larger numbers take longer time to factorize, which can be observed by the test below.   
**NOTE: THE FOLLOWING CODE MAY TAKE A LONG TIME TO FINISH. IF YOU DONT WANT TO WAIT, USE THE MENU Kernel->Restart TO STOP IT**

In [3]:
# define a timer function
def timer(func, *args): 
    import time
    start_time = time.time()
    func(*args)
    end_time = time.time()
    return end_time-start_time

### USE GMP version instead for large integers
import RSACipherGMP as gmp

for bits in range(6,30,2):
    prime = gmp.next_prime(2**bits)
    n = 13*prime 
    duration = timer(gmp.fermat_factors, n)
    print(f"time to factorize {n}=13x{prime} is {duration}")

time to factorize 871=13x67 is 3.0994415283203125e-05
time to factorize 3341=13x257 is 2.288818359375e-05
time to factorize 13403=13x1031 is 0.00010800361633300781
time to factorize 53287=13x4099 is 0.0006000995635986328
time to factorize 213343=13x16411 is 0.0020880699157714844
time to factorize 851981=13x65537 is 0.00823211669921875
time to factorize 3407911=13x262147 is 0.03361201286315918
time to factorize 13631579=13x1048583 is 0.13180017471313477
time to factorize 54526147=13x4194319 is 0.5027520656585693
time to factorize 218104367=13x16777259 is 2.0302011966705322
time to factorize 872415427=13x67108879 is 8.01463508605957
time to factorize 3489660967=13x268435459 is 32.25048804283142
